In [453]:
import pandas as pd
import numpy as np

In [454]:
vdf = pd.read_csv('fruitvegprices-2017_2022.csv')

In [455]:
vdf['date']=pd.to_datetime(vdf['date'])

In [456]:
vdf['year']=vdf['date'].dt.year


In [457]:
vdf['Month']=vdf['date'].dt.month

In [458]:
vdf

,category,item,variety,date,price,unit,year,Month
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg,2022,3
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg,2022,3
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg,2022,3
3,fruit,apples,braeburn,2022-03-11,1.05,kg,2022,3
4,fruit,apples,gala,2022-03-11,1.03,kg,2022,3
...,...,...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem,2017,11
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem,2017,11
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem,2017,11
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem,2017,11


## Ejercicios

### Ejercicio uno: realice un analisis de valores nulos

En caso de existir valores nulos elimine esos registros

## primera forma

In [459]:
vdf.isnull().sum()

category    0
item        0
variety     0
date        0
price       0
unit        0
year        0
Month       0
dtype: int64

## segunda forma

In [460]:
for s in vdf.columns:
    print(s)
    print(np.sum(vdf[s].isna()))

category
0
item
0
variety
0
date
0
price
0
unit
0
year
0
Month
0


## nota: por lo tanto no existen valores nulos en el dataframe.

### Ejercicio dos: Por categoria, cuantos distintos items hay y cuantos distintos variety hay?

El resultado debe de tener el siguiente formato de presentacion:

In [461]:
pd.DataFrame({
    "category" : ['cut_flowers','fruit']
    , "item_count" : [9,10]
    , "variety_Count" : [10,21]
})

,category,item_count,variety_Count
0,cut_flowers,9,10
1,fruit,10,21


Como nota en un mismo dataframe mostramos todo lo solicitado, usted debe de mostrar todas las categorias

In [462]:
vdf.loc[:,['category','item','variety']]

,category,item,variety
0,fruit,apples,bramleys_seedling
1,fruit,apples,coxs_orange_group
2,fruit,apples,egremont_russet
3,fruit,apples,braeburn
4,fruit,apples,gala
...,...,...,...
9642,cut_flowers,alstromeria,indoor
9643,cut_flowers,chrysanthemum,indoor_spray
9644,cut_flowers,lillies,oriental
9645,cut_flowers,narcissus,indoor


In [463]:
vdf.groupby(['category',], as_index=False).agg({ 'variety':['count'],'item':['count']  })

,category,variety,item
,,count,count
0,cut_flowers,342,342
1,fruit,1992,1992
2,pot_plants,49,49
3,vegetable,7264,7264


### Ejercicio tres: Para la combinacion item, variety muestre el ultimo valor registrado por año.

Ejemplo de como debe de mostrar su respuesta

In [464]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year" : [2022]
    , "last_price" :  [3.2]
})

,item,variety,year,last_price
0,Manzana,Gala,2022,3.2


In [465]:
lastValue=vdf.loc[:,['item','variety','year','price']]

In [466]:
lastValue=lastValue.sort_values(['year'],ascending=True)

In [467]:
lastValue['rank']=lastValue.groupby(['item','variety','year','price'],as_index=False)['year'].rank(method='first', ascending=False)

In [468]:
lastValue=lastValue.loc[lastValue['rank']==1.0,:]

In [469]:
lastValue=lastValue.drop(['rank'],axis=1)

In [470]:
lastValue.columns=['item','variety','year','last_price']

In [471]:
lastValue.drop_duplicates(['year'])

,item,variety,year,last_price
9646,cyclamen,13_cm,2017,0.75
7701,lillies,oriental,2018,0.65
5523,cabbage,round_green_other,2019,0.50
3200,lettuce,butterhead_indoor,2020,0.39
1034,spinach_leaf,loose_bunches,2021,1.42
103,apples,other_late_season,2022,1.59


### Ejercicio cuatro: Segun el ejercicio tres, que tanto aumento o disminuyo el precio de cada item-variety 

Ejemplo de como se debe de mostrar su respuesta

In [472]:
pd.DataFrame({
    "item" : ['Manzana','Manzana','Manzana']
    , "variety" : ['Gala','Gala','Gala']
    , "year" : [2020,2021,2022]
    , "last_price" :  [2.7,2.5,3.2]
    , "delta" : [0, (2.5-2.7), (3.2-2.5)]
})

,item,variety,year,last_price,delta
0,Manzana,Gala,2020,2.7,0.0
1,Manzana,Gala,2021,2.5,-0.2
2,Manzana,Gala,2022,3.2,0.7


Segun nuestro ejemplo dummy para 2021 estuvo 0.2 centavos mas barato y en 2022 estuvo 0.7 centavos caro

In [473]:
lastValue=lastValue.sort_values(['item','variety'])

In [474]:
lastValue['delta']=lastValue.groupby(['item','variety','year'],as_index=False)['last_price'].shift(1)

In [475]:
lastValue=lastValue.fillna(0)

In [476]:
lastValue['AumentoDisminución']=(lastValue['last_price']-lastValue['delta'])

In [477]:
lastValue

,item,variety,year,last_price,delta,AumentoDisminución
9387,alstromeria,indoor,2017,0.28,0.00,0.28
9434,alstromeria,indoor,2017,0.29,0.28,0.01
9304,alstromeria,indoor,2017,0.27,0.29,-0.02
9587,alstromeria,indoor,2017,0.26,0.27,-0.01
7698,alstromeria,indoor,2018,0.32,0.00,0.32
...,...,...,...,...,...,...
6282,watercress,pillow_packs,2019,7.92,9.45,-1.53
6363,watercress,pillow_packs,2019,9.68,7.92,1.76
6157,watercress,pillow_packs,2019,9.37,9.68,-0.31
6243,watercress,pillow_packs,2019,9.95,9.37,0.58


### Ejercicio cinco: Cual es precio promedio por item, variety

Ejemplo de como se debe de mostrar su respuesta

In [478]:
pd.DataFrame({
    "item" : ['Manzana']
    , "variety" : ['Gala']
    , "year-month" : ['2021-01']   
    , "avg-price" : [0.5]
})

,item,variety,year-month,avg-price
0,Manzana,Gala,2021-01,0.5


In [479]:
vdf

,category,item,variety,date,price,unit,year,Month
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg,2022,3
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg,2022,3
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg,2022,3
3,fruit,apples,braeburn,2022-03-11,1.05,kg,2022,3
4,fruit,apples,gala,2022-03-11,1.03,kg,2022,3
...,...,...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem,2017,11
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem,2017,11
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem,2017,11
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem,2017,11


In [480]:
vdf['year'] = vdf['year'].apply(str)

In [481]:
vdf['Month'] = vdf['Month'].apply(str)

In [482]:
vdf['year-month'] = vdf['year'].str.cat(vdf['Month'], sep ="-")

In [163]:
vdf

,category,item,variety,date,price,unit,year,Month,year-month
0,fruit,apples,bramleys_seedling,2022-03-11,2.05,kg,2022,3,2022-3
1,fruit,apples,coxs_orange_group,2022-03-11,1.22,kg,2022,3,2022-3
2,fruit,apples,egremont_russet,2022-03-11,1.14,kg,2022,3,2022-3
3,fruit,apples,braeburn,2022-03-11,1.05,kg,2022,3,2022-3
4,fruit,apples,gala,2022-03-11,1.03,kg,2022,3,2022-3
...,...,...,...,...,...,...,...,...,...
9642,cut_flowers,alstromeria,indoor,2017-11-03,0.27,stem,2017,11,2017-11
9643,cut_flowers,chrysanthemum,indoor_spray,2017-11-03,0.22,stem,2017,11,2017-11
9644,cut_flowers,lillies,oriental,2017-11-03,0.70,stem,2017,11,2017-11
9645,cut_flowers,narcissus,indoor,2017-11-03,0.06,stem,2017,11,2017-11


In [483]:
s=vdf.groupby(['item','variety','year-month'], as_index=False).agg({ 'price':['mean'] })

In [484]:
s.columns=s.columns.droplevel(1)

In [485]:
s.columns = ['item', 'variety', 'year-month', 'avg_price']

In [486]:
s

,item,variety,year-month,avg_price
0,alstromeria,indoor,2017-11,0.2700
1,alstromeria,indoor,2017-12,0.2800
2,alstromeria,indoor,2018-1,0.3100
3,alstromeria,indoor,2018-10,0.2350
4,alstromeria,indoor,2018-11,0.2580
...,...,...,...,...
2580,watercress,pillow_packs,2019-4,9.4750
2581,watercress,pillow_packs,2019-5,9.5000
2582,watercress,pillow_packs,2019-6,9.4900
2583,watercress,pillow_packs,2019-7,9.4975


### Ejercicio seis: muestre por category,item,variety que tanto % ha aumentado su precio

Ejemplo: si para las manzanas gala el primer precio que se registro fue 2.00 y el ultimo precio que se registro es 4.00 este aumento un 100%

In [487]:
pd.DataFrame({
    "category": ["fruta","fruta"]
    ,"item" : ['Manzana','Manzana']
    , "variety" : ['Gala','egremont_russet']
    , "percentage-change-between-first-and-last-price" : [200,150]       
})

,category,item,variety,percentage-change-between-first-and-last-price
0,fruta,Manzana,Gala,200
1,fruta,Manzana,egremont_russet,150


In [488]:
x=vdf.loc[:,['category','item','variety','price']]

In [489]:
x=x.groupby(['category','item','variety'], as_index=False).agg({ 'price':['max','min'] })

In [490]:
x.columns=x.columns.droplevel(1)

In [491]:
x.columns=['category','variety','item','maxPrice','minPrice']

In [492]:
x['percentage']=(x['maxPrice']-x['minPrice'])/100

In [310]:
x

,category,variety,item,maxPrice,minPrice,percentage
0,cut_flowers,alstromeria,indoor,0.34,0.23,0.0011
1,cut_flowers,chrysanthemum,indoor_spray,0.25,0.17,0.0008
2,cut_flowers,gladioli,all_varieties,0.33,0.10,0.0023
3,cut_flowers,lillies,oriental,0.99,0.50,0.0049
4,cut_flowers,narcissus,indoor,0.09,0.04,0.0005
...,...,...,...,...,...,...
80,vegetable,tomatoes,plum,4.50,0.55,0.0395
81,vegetable,tomatoes,round,1.67,0.45,0.0122
82,vegetable,tomatoes,vine,4.45,0.71,0.0374
83,vegetable,turnip,turnip,2.10,0.55,0.0155


### Ejercicio siete: Segun el ejercicio seis por category cual es el item,variety con mayor aumento de crecimiento 

Segun el ejemplo dummy que se utilizo en el ejercicio seis para la category fruta se mostraria la manzana gala unicamente

In [493]:
x.groupby(['category','item','variety'], as_index=False).agg({'percentage':['max']})

,category,item,variety,percentage
,,,,max
0,cut_flowers,all_varieties,gladioli,0.0023
1,cut_flowers,indoor,alstromeria,0.0011
2,cut_flowers,indoor,narcissus,0.0005
3,cut_flowers,indoor_spray,chrysanthemum,0.0008
4,cut_flowers,oriental,lillies,0.0049
...,...,...,...,...
80,vegetable,trimmed,leeks,0.0126
81,vegetable,turnip,turnip,0.0155
82,vegetable,vine,tomatoes,0.0374
